In [1]:
import sys
import os
import glob
import re

import pandas as pd
import pandas_profiling as pdp
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.svm import SVR
from sklearn import metrics
import optuna

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../processed_data/train_v2.csv')
df = df.loc[:,['id', 'age','area','floor', 'structure', 'max_floor',
       'room_num', 'L', 'D', 'K', 'S', 'north', 'y']]
df.head(2)

,id,age,area,floor,structure,max_floor,room_num,L,D,K,S,north,y
0,1,117,20.01,1,7,12,1,0,0,1,0,0,75000
1,2,538,16.50,5,2,10,1,0,0,0,0,0,76000


In [3]:
X, y = df.loc[:,'age':'north'], df['y']

In [4]:
X,y

(       age   area  floor  structure  max_floor  room_num  L  D  K  S  north
 0      117  20.01      1          7         12         1  0  0  1  0      0
 1      538  16.50      5          2         10         1  0  0  0  0      0
 2      102  22.05     12          7         15         1  0  0  1  0      0
 3      352  60.48      3          7          4         2  1  1  1  0      0
 4      379  39.66      1          1          2         2  0  1  1  0      0
 ...    ...    ...    ...        ...        ...       ... .. .. .. ..    ...
 31465  360  37.90      1          2          3         2  0  1  1  0      0
 31466  475  12.04      4          7          4         1  0  0  0  0      0
 31467  550  60.00      3          2          5         3  0  1  1  0      0
 31468  132  29.59      1          1          2         1  0  1  1  0      0
 31469  722  25.00      2          2          3         1  0  0  1  0      0
 
 [31470 rows x 11 columns], 0         75000
 1         76000
 2        110

In [25]:
def objective(trial):  
    params = {
        'kernel': trial.suggest_categorical('kernel', ['linear','rbf','sigmoid']),
        'C': trial.suggest_loguniform('C', 1e+1, 1e+2),
        'gamma': trial.suggest_loguniform('gamma', 1e-10, 1e-1),
    }
    
    mdl = SVR(**params)
    kfold = KFold(n_splits=5, random_state=0,shuffle=True)
    scores = cross_val_score(mdl, X, y, cv=kfold,scoring='neg_mean_squared_error')
    score = np.sqrt(abs(scores)).mean()
#     print(score)
    return score

In [26]:
# sorted(metrics.SCORERS.keys())

In [27]:
study = optuna.create_study()

In [ ]:
optuna.logging.enable_default_handler()
study.optimize(objective, n_trials=10)

[I 2019-09-28 05:50:20,620] Finished trial#0 resulted in value: 75554.62164037733. Current best value is 75554.62164037733 with parameters: {'kernel': 'sigmoid', 'C': 25.518195125665155, 'gamma': 0.09683665811994953}.
[I 2019-09-28 05:52:43,625] Finished trial#1 resulted in value: 75522.98324018446. Current best value is 75522.98324018446 with parameters: {'kernel': 'rbf', 'C': 10.532492243053735, 'gamma': 0.054490305746458356}.
[I 2019-09-28 05:55:59,888] Finished trial#2 resulted in value: 43167.76491215391. Current best value is 43167.76491215391 with parameters: {'kernel': 'linear', 'C': 11.375456908332545, 'gamma': 5.110606796007172e-07}.
[I 2019-09-28 05:58:51,394] Finished trial#3 resulted in value: 75153.34672924105. Current best value is 43167.76491215391 with parameters: {'kernel': 'linear', 'C': 11.375456908332545, 'gamma': 5.110606796007172e-07}.


In [9]:
print(study.best_trial)

ValueError: No trials are completed yet.

In [ ]:
study.best_params

In [ ]:
mdl = SVR(**study.best_params)
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
mdl.fit(X_train, y_train)

pred_train = mdl.predict(X_train)
accuracy_train = accuracy_score(y_train, pred_train)
pred_test = mdl.predict(X_test)
accuracy_test = accuracy_score(y_test, pred_test)
print('train score :',accuracy_train)
print('test score :',accuracy_test)

In [ ]:
mdl = SVR(**study.best_params)
mdl

In [ ]:
mdl.fit(X, y)

In [15]:
test_df = pd.read_csv('~/Workspace/learning/signate/mynavi_2019/processed_data/test_v2.csv')
input_df = test_df.loc[:,['age','area','floor', 'structure', 'max_floor','room_num', 'L', 'D', 'K', 'S', 'north']]

In [19]:
input_df.isnull().sum()

age          0
area         0
floor        0
structure    0
max_floor    0
room_num     0
L            0
D            0
K            0
S            0
north        0
dtype: int64

In [17]:
test_row = pd.read_csv('../data/test.csv')
test_row.head(3)

,id,所在地,アクセス,間取り,築年数,方角,面積,所在階,バス・トイレ,キッチン,放送・通信,室内設備,駐車場,周辺環境,建物構造,契約期間
0,31471,東京都世田谷区深沢５丁目1-27,東急大井町線\t等々力駅\t徒歩15分\t\t東急田園都市線\t桜新町駅\t徒歩24分\t\...,2LDK,49年0ヶ月,南,50.22m2,8階／8階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t追焚機能／\t温水洗浄便座／...,ガスコンロ／\tコンロ3口／\tシステムキッチン\t／\t給湯／\tL字キッチン,インターネット対応,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\tエレベーター\t公営水...,駐輪場\t空有\t駐車場\t無\tバイク置き場\t無,【コンビニ】 74m\t【スーパー】 215m\t【病院】 313m\t【小学校】 219m...,鉄骨造,NaN
1,31472,東京都目黒区八雲１丁目11-8,東急東横線\t都立大学駅\t徒歩6分\t\t東急東横線\t自由が丘駅\t徒歩17分\t\t東...,1R,0年2ヶ月,南東,20.88m2,3階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t浴室乾燥機\t／\t温水洗浄...,ガスコンロ／\tコンロ2口／\tシステムキッチン\t／\t給湯,インターネット対応／\tCSアンテナ／\tBSアンテナ／\tインターネット使用料無料,エアコン付\tシューズボックス／\tフローリング／\t室内洗濯機置場\t公営水道／\t下水／...,"駐車場\t近隣\t34,000円(税込)\t距離200m\t駐輪場\t無\tバイク置き場\t無",【スーパー】 128m\t【スーパー】 313m\t【コンビニ】 374m\t【コンビニ】 ...,RC（鉄筋コンクリート）,2年間
2,31473,東京都豊島区池袋本町２丁目22-2,東武東上線\t北池袋駅\t徒歩10分\t\t埼京線\t板橋駅\t徒歩16分\t\t山手線\t...,1K,23年4ヶ月,南東,26.93m2,1階／4階建,専用バス／\t専用トイレ／\tバス・トイレ別／\tシャワー／\t温水洗浄便座,ガスコンロ／\t給湯,インターネット対応／\t光ファイバー／\tCATV,エアコン付\tシューズボックス／\tバルコニー／\tフローリング／\t2面採光／\t室内洗濯...,"駐車場\t空有\t5,000円(税込)\t(敷金あり\t1ヶ月)\t駐輪場\t空有\tバイク...",【スーパー】 529m\t【スーパー】 810m\t【コンビニ】 316m\t【コンビニ】 ...,鉄骨造,2年間


In [18]:
test_row['建物構造'].unique()

array(['鉄骨造', 'RC（鉄筋コンクリート）', '木造', 'SRC（鉄骨鉄筋コンクリート）', '軽量鉄骨',
       'ALC（軽量気泡コンクリート）', 'その他', 'HPC（プレキャスト・コンクリート（重量鉄骨））',
       'PC（プレキャスト・コンクリート（鉄筋コンクリート））', '鉄筋ブロック', 'ブロック'], dtype=object)